# Getting Started Containers
## Overview

Virtual Machines have their own kernel on the HyperVisor, while a container runtime uses the host machine kernel

System containers are used as a foundation to build your own container images
* **alpine**
* **ubuntu**

These are not meant to be replacements of a virtual machine

Application containers are used to simply start an application

Containers rely on features of the Linux Kernel
* **namespaces** and **chroot** -- limit what can be accessed and seen, resource isolation
* **cgroups** -- resource limitation
* **SELinux** -- security

Docker Desktop provides an easy way to run containers on Windows or Mac

Linux containers are ordinary processes on a Linux system, isolated from other processes. They are all part of a cgroup.
* `cat /proc/[PID]/cgroup`
* `cat /proc/self/attr/current` -- SELinux labels
* `ls /proc/[PID]/ns` -- list of namespaces

A container is different in that it is started from a container image

The container runtime is the operating system part that manages cgroups, SELinux labels and namespaces and starts the container from an image

The container orchestration layer allows for containers to work together. Similar to microservice idea of developing minimal pieces of code and joining them.
* Kubernetes -- the standard orchestration platform
* OpenShift
* Rancher

A container is a running instance of an image. An image is the application code, language runtime and libraries needed to run the application.

A container image would also include external libs often provided by host system. Because of this it may be tied to a certain linux distro

The image is a read-only instance of the application. A writable layer is added so that changes can be stored while working with the container.

A container image is basically a tar ball containing:
* Container root file system - presented as a mount namespace
* Metadata - JSON file that specifiis how to run the root file system and the settings required
* The entrypoint application, environment variables, etc.

Container images are layered. You can install additional content, add a new JSON file, and store the differences in a new TAR file

Container images are shared through public registries or shared so they can be built easily --such as with a Dockerfile.

Docker images are made of a series of filesystem layers. Each layer adds, remove, or modifies files from the preceding layer in the filesystem. It is an example of an overlay filesystem.

Apart from its different layers, container images have a container configuration file that provides instructions on how to run the container.

The starting layer is often a small system image like alpine. The Ruby application layer would sit on top of the system image layer. Since there are different layers, the setting for both can be managed independently. You can also add your own application on top of these layers.



# Registries
A container Registry helps with the distribution of images. You can also manually distribute images with tar balls, but this is not  recommended.

Remote Registries are common, such as:
* DockerHub registry -- https:hub.docker.com
* Quay -- https://quay.io --Used for RedHat images. Open Source.

RHEL registry location `/etc/containers/registries.conf` is used to tell RHEL where to go
* *registry.access.redhat.com* - no authentication but deprecated
* *registry.redhat.io* - requires authentication

Setting up your own local private registry is also possible.

There may be a need for authenticated registry access. This allows for added functionality and allows for more downloads of images.
- `podman login` to authenticate to the registry

Downloading an image. Can specify the exact registry you want to pull from. Can also specify the tag to download a specific version of the image
- `podman pull nginx` 

Include registry in pull command to ensure where you are getting it
* `podman pull registry.redhat.io/ubi8/ubi`

Search for an image interested in using
* `docker search nginx`
* `docker serach --filter "is-official=true" ubuntu` Will show official images

### Creating Private Registries

On CentOs:
* `dnf install docker-distribution`
* `systemctl enable --name docker-distribution`
* Config is in `/etc/docker-distribution/registry/config.yml`
Registry service listens on port 5000, open it in firewall
`sudo firewall-cmd --add-port 5000/tcp`

On Ubuntu:
* `docker run -d -p 5000:5000 --restart=always --name registry registry:latest`
* `sudo ufw allow 5000/tcp`

Need to tag image to push it to your own image registry. Will save tagged image locally.
* `docker tag fedora:latest localhost:5000/myfedora`

Push to local registry
* `docker push localhost:5000/myfedora`
* `docker rmi fedora` --Remove local image
* `docker rmi localhost:5000/myfedora` --Remove tagged image.

Pull from local registry to get image again from local
* `docker pull localhost:5000/myfedora1`

Can create Automatic Image Builds from Git Repo. Need to setup webhooks.
* After creating changes to DockerFile on Git will upload and build to Docker Registry the image
    * Pull of Docker Registry image will get the update you committed to Git

# Docker Tools

Docker Desktop uses host OS virtualization features to run a Docker virtual machine -- Hyper-Visor. 

Docker Toolbox is an older technology, for older machines, using VirtualBox virtualization platform.

Docker Desktop cannot route traffic to containers so you cannot access an exposed port.

# Container Runtime and Engine

Container engine takes a container image and turns it into a container and is comprised of:
* runtime
* CLI tool
* sometimes daemon

Container runtime is a part of the Container engine and sets up the container lifecycle and image management. 

Commonly used runtimes:
* CRI-O -- Red Hat
* containerd -- Docker

The Container runtime provides the mount point, communicates with the kernel, sets up cgroup and namespaces, etc.

*runC* is a lightweight universal container runtime. It is the default runtime defined by OCI. It focuses on creating containers and is included in CRI-O and *containerd*.

*containerd* manages the container lifecycle and executes running *runc*. It extracts the image and makes it a runnable container

*containerd-shim* process allows for daemonless containers, it takes over after runc has started the container

Docker CLI is the docker command and communicates with the dockerd API. *dockerd* is the Docker daemon that listens for API requests and utilizes containerd

# Common Commands

To view all images currently avaiable
* `docker images`
    * `docker image ls`

Run in attached mode and set up an sh shell in busybox with interactive terminal.
* `docker run -it busybox /bin/sh`

To disconnect from attached mode use **Cntrl-P, Cntrl-Q**. Exiting while in attached mode would kill the app.

Run in detached mode with -d option
* `docker run -d`

Attach to container started in detached mode
* `docker attach [container]`

See an overview of all containers started. May not currently be active.
* `docker ps -a, docker ps --all`
    * `docker container ls [-a]`

See active container processes
* `docker ps`
    * `docker ps -s` show size information

Lifecycle commands
* `docker pull [image]` downloads the image from a registry
* `docker run [image]` runs a command in a new container and pulls the image if needed and starting it
    * `docker container run` The preferred way of running
    * Run bash terminal in interactive mode for ubuntu and remove afterward calling it mycontainer
        * `docker run --rm --name=mycontainer -it ubuntu:latest /bin/bash`
* `docker create [image]` stages the container but doesn't start it
* `docker start [containerID]` starts the staged container
* `docker stop [containerID]` kill -15
* `docker kill [containerID]` kill -9
* `docker pause [containerID]` pause the container
* `docker unpause [containerID]` unpause the container
* `docker rm [containerID]`, `docker rmi [containerID]` removes the container

Search for an image interested in using
* `docker search nginx`
* `docker search --filter "is-official=true" ubuntu` Will show official images

Review Commands
* `docker inspect` Review details of running containers. JSON can give details of current configuration and necessary requirements to run.
* `docker logs` Access primary application STDOUT
* `docker stats` Linus top-like interface
* `docker top [containerid]` process utilization of the container

Troubleshooting
* Use `docker ps -a` to see if container quit unexpectedly
* Containers that don't start ofen have issue with entrypoint application. The last option to run should always be the entrypoint.
* Review the logs with `docker logs <containerid>` to connect to STDOUT

# Container Process and Resource Limitation

On host OS a Docker container is just a running process

A container does not have access to the the running parts of other containers

Using `ps aux` all the running Docker containers show as Linux processes

Can see docker listening on a socket with containerd
* `ps fax | grep docker -A 3`

Containers are Linux processes and have full access to system resources

Linux kernel provides Cgroups that put a limit on this

Set a hard memory limit
* `docker run -d -p 8081:80 --memory="128m" nginx`

Set a soft limit that is enforced if a memory shortage exists
* `docker run -d -p 8082:80 --memory-reservation="256m" nginx`

Docker has Cgroup notion of CPU Shares. This is a relative weigth.

All containers get a CPU share weight of 1024

Run container on 4 CPU cores with relative CPU shares set to %50 (Half of 1024=512)
* `docker run -it --rm -c 512 mycontainer --cpus 4`

Containers can be pinned to a specific core. Run Container on 2 cores, specified as 0 and 2
* `docker run -it --rm --cpuset-cpus=0,2 mycontainer --cpus 2`

Can inspect current memory restriction with "Memory" parameter
* `docker inspect mycontainer`

Specify the core to run so that will be partial on that core rather than expanding across all cores
* `docker run -dit --rm -c 2048 --cpuset-cpus=0 --cpus=1 busybox dd if=/dev/zero of=/dev/null`
* `docker run -dit --rm -c 512 --cpuset-cpus=0 --cpus=1 busybox dd if=/dev/zero of=/dev/null`




# Running the Container - EntryPoint

When the container is started the entrypoint command is started as the defaul command

`docker inspect [image]` and look for "Cmd" to find which command this is

Can overwrite the default command and specify a new command while running

The following will not run nginx but instead open an sh terminal to the container
* `docker run --name mynginx -it nginx sh`

May need to specify Environment varaible in running the container. May be able to see needed variables in JSON with inspect.
* `docker run -d -e MARIADB_ROOT_PASSWORD=secret mariadb`

# Docker Compose

Docker-compose is a declarative approach to start Docker containers from a manifest file written in YAML

Create a **docker.compose.yml** file and include parameters typically used on the command line while starting a Docker container.

To run your docker compose in detached mode
* `docker-compose up -d command` (detached mode)

To stop the container
* `docker-compose down`

Kubernetes replaces the need for using docker-compose

podman-compose is a podman alternative to docker-compose

Resources
* https://docs.oracle.com/en/learn/podman-compose/#install-docker-compose 
* https://fedoramagazine.org/use-docker-compose-with-podman-to-orchestrate-containers-on-fedora/

Setup Docker Compose with Rootless Podman Socket
* `systemctl --user enable podman.socket`
* `systemctl --user start podman.socket`
* `systemctl --user status podman.socket`
* `export DOCKER_HOST=unix:///run/user/$UID/podman/podman.sock`

Setup Docker Compose with Rootful Podman Socket
* `sudo systemctl enable --now podman.socket`
* `sudo systemctl status podman.socket`

Test the Socket
* `sudo curl -H "Content-Type: application/json" --unix-socket /var/run/docker.sock http://localhost/_ping`


# Podman - Alternative to Docker

RHEL 8 Docker Implementation
* OpenShift is a RHEL product for managing containers in an enterprise environment. An orchestration product.
* RHEL 8 does not support Docker instead it supports Podman
* No docker daemon is required and runs daemon-less

RHEL Docker Tools
* **podman**: docker alternative. Direct management of pods and container images
* **buildah**: building, pushing, and signing container images
* **skopeo**: copying, inspecting, and signing images

CRI-O- container runtime interface: container engine that provides container run and build features to podman and buildah

Install Podman
* `dnf module install container-tools` (May already be installed)
* `dnf install podman-docker` (Docker like syntax with podman)

Containers in podman can be started by root user or unprivileged user

In both cases the container is started with its own user namespace and in either case the user has its own */etc/password*, *etc/shadow files* as well as container images.

To run containers as a different UID inside the container use the -u username option

Can also use container images that specify which user to use

When starting a root container process with a different UID as on the host the same UID will be used as the owner of the process on the host.

When using a rootless container the container is started by a non-priveleged user while processes within the container may still have root privideges
* These processes don't have any host access privileges as it is a rootless container.
* This limits common functionality on the host such as the option to bind to a privieged port.

You can use a rootless container with a non-root user in the container which is the most secure option.

Options to run Containers
* root container running a root process in the container
* root container running a non-root process in the container
* rootless container running a root process in the container
* rootless container running a non-root process in the container

While starting a rootless container that runs a non-root process a mapping must be made between the user inside the container and a user on the host

By default Podman uses the same UID on both the host and the container

A UID map file can be used to map between host user and container user

Rootful containers running a Root process shows on Host system as Root
- `sudo podman run -it quay.io/quary/busybox sh`
- `# whoami`
- `root`
- `# sleep 1000 &`
- `# ps aux` --- Owner is root
- [Disconnect]
- `ps aux | grep sleep` --Owner is root on the host system of sleep 1000

Rootless container running Root process shows on Host system as that user

Root container running a non-root process. Same UID used on host as well.
* `sudo podman run -it -u 27 quay.io/quary/busybox sh`
    * Specifying user 27, will run as user 27 on the container and host.
    * numeric userid will not be mapped to an existing user
    * Should be user where that numeric userid is not already existing or process will run as that user which may not be wanted and cause unwanted effects.

Rootless container running a non-root process. Container UID will be mapped dynamically to new UID on host.
* Again specifying user 27 userid
    * In this case 27 is not used by host, instead using a mapped userid like 100026
    * In a rootless container can't map to just any userid so is generated dynamically

Containers can be shown as rootful or rootless
* `podman ps` --Shows rootless containers
* `sudo podman ps` -- Shows rootful containers

Using inspect on rootless containers shows no Network Settings. On rootful containers will see Network Settings. (Use sudo for inspect)

Rootless Container UID mappings
* `/etc/subuid` on host OS has userIDs that can be used by the rootless container to map against these userIDs on the host OS

This mapping can also be manually specified

The `/proc/[PID]/uid_map` file shows container mappings used inside the container

Shows mapping of user inside the container against the host user
* `podman top -l user huser args`

File ownership
* If user 27 in container creates a file the file will be owned by the generated userID on host OS
* To see file owner with UID used inside the container. Useful in analyzing file access issues while running rootless containers
    * `podman unshare ls -al /volume/mount/point`

# Building Image Options

Different options exist to build an image
* Mounting the root directory of a container and modifying that
* Using native buildah commands
* From Dockerfile
* From scratch

## Buildah Image Creation

Buildah Tool
* A tool that helps in building images
* Not preferred tool for actually running containers
* Doesn't need a container runtime daemon but uses runc

Using buildah native commands make it easy to script the entire build process
* `buildah bud` -- build an image from a Dockerfile
* `buildah from [imagename]` -- builds an image from another image
* `buildah from scratch` -- Allows you to build from scratch
* `buildah inspect` -- Shows container or image metadata
* `buildah mount` -- Mounts the container root FS
* `buildah commit` -- Uses updated contents of a container root FS and an FS layer to commit content to a new image
* `buildah rmi/rm` remove image or container
* `buildah unmoun`t -- unmounts container if you have mounted it
* `buildah images` -- list of images
* `buildah containers` -- list of containers

Images are what a container is started from. A container is a running instance of an image.

Docker images are immutable with each modification adding an extra layer to the pre-existing layers.

The container sees it as a single virutal file system by using UnionFS or another driver.

`docker history [imageid], docker history image:tag` -- Shows the different layers in the image

Each modification adds an image layer.

Creating image Options
* From a running container, modifications are applied and docker commands are used to write modifications.
* Using a Dockerfile - Contains instructions for building an image. Each instruction adds a new layer to the image. More control over what files are added to which layer.
* Use buildah for RHEL

A child image is an image that is created from a parent iamge and incorporates everything in the parent image and adds to it.

## Dockerfile Creation

Dockerfile Process
* First create a working directory -- each project should have its own project directory
* Next write the Docker file
* Finally build the image using Docker command

Dockerfile starts with FROM, identifying the base image to use

* Instructions are executed in the base image and are executed in the order specified.
* Each Dockerfile instruction runs in a independent container using an intermediate image built from the previous command. Adding multiple instructions results in multiple layers
* You should try to minimize the amount of layers.

Identify the base image to use
* `FROM centos`

Name of the person that maintains the image
* `MAINTAINER Sander`

`LABEL` is a key-value pair optionally used for identification

`Add` can move files from the project directory to the image. `COPY` also works like `ADD` but `ADD` has more flexibility. `ADD` can get contents of a tar file inside the container and can fetch files from a URL.
* `ADD ./sander.repo /etc/yum.repos.do/`

`USER` specifies the username for RUN, CMD, and ENTRYPOINT instructions

`RUN` Execute Command
* Can use RUN wget to fetch file from url and delete afterward
* `RUN yum -y update && ...`

`EXPOSE` has metadata only information on where the image should run

`ENV` defines environment variables to be used within the container

SPECIFY `CMD` - can have the default command and arguments to the ENTRYPOINT command
* `CMD ["/user/bin/nmap","-sn","127.17.0.0/24"]`

`ENTRYPOINT` is the default command to be processed
* If not specified /bin/sh -c is executed as the default command
* `CMD` provides arguments to the ENTRYPOINT command

If the default command is specified using CMD instead of ENTRYPOINT, the command is executed as an argument to the default entrypoint sh -c, which can give unexpected results.

If arguments to the command are specified within the ENTRYPOINT they cannot be overwritten so they should be passed in with the CMD section instead.

Options like `ADD`, `COPY`, `ENTRYPOINT`, `CMD` are used in shell form and in exec form

Shell form is presented as a list of items
* `ADD /my/file /mydir`
* `ENTRYPOINT /usr/bin/nmap`

Exec form is a JSON array of items. This is the preferred way as shell form wraps the command in a shell. Otherwise, can create unnecessary shell processes.
* `ADD ["/my/file", "mydir"]`
* `ENTRYPOINT["/usr/bin/nmap"]`

Try not running multiple `RUN` commands as this creates a new layer. Attempt to use one RUN command with multiple && connecting operations.

`RUN yum --disablerepo=* --enablerepo="rhel17-server-rpms" && yum update -y && yum install -y nginx`

Build an image using docker build taking in an image name and working directory with Docker file. Tag optional.
* `docker build -t myimage::latest .` -- If no tag specified will use latest

Ensure complete procedure run again
* `docker build --no-cache -t nmap .`

On Docker hub can find good examples of Docker files

## Other Image Creation Options

Can Save container changes with Docker commit to an image
* `docker commit -m "custom web server" -a "Sander" myapache myapache` (-m commit message -a author, docker commit [options] [container] [image])

Save image as tar ball (Not recommended)
* `docker save -o myapache.tar myapache`
Load tarball and import as an image
* `docker load -i myapache.tarm`


# Image Best Practices

Images are multi-layered
* Should have a minimal base layer if possible as data will still be in base layer
* Use specific base images (meant for base) and avoid generic os images like ubuntu image as this is likely too big.
* Consider using multistage builds where a Dockerfile goes through different stages with only the last stage kept.
* Minimize the number of RUN commands in the Dockerfile as each run command creates its own layer.
* Can create your own base images if you have many images with a great deal in common.
* In Kubernetes can use Secrets and Configs to keep data out of the images.

# Storage Solutions

Storage Options
- Use a bind mount to a file system on the host OS
- Connect to an external (SAN or cloud-based) persistent storage solution. May need ochestration layer.
- To connect to external storage, volumes are used, specific drivers can specify which volume type to connect to.
- For temporary data, tmpfs can be used

Storage drivers allow for writing data in the writable layer of the container

For local use, the local driver and the sshfs driver are available

Different storage drivers are available for local use.

Storagedrivers are set in `/etc/docker/daemon.json`
````
{
	"storage-driver":"devicemapper"
}
````

CoW Storage Stategy
* Only modifications are stored in the writable filesystem layer


Bind mount storage
- Container mounts a directory or file from the host OS into the container
- host directory can be automatically created with -v option, or --mount
- The host OS fully controls access to the file

bind mount to `/dev/log` journalctl of host should show hello with journalctl
* `podman run --name=mycontainer -v /dev/log:/dev/log --rm ubuntu:latest logger hello`

Can see under list of "Mounts" the bind mount
* `docker inspect [containername]` to verify

Show mapping of host folder to container folder
* `docker run --rm -dit --name=bind2 -v "$(pwd)"/bind2:app nginx:latest`
* Place file bond.txt in bind2 folder
* `docker exec bind2 ls -l /app` 
    * This shows bond.txt in container folder which matches what is in Host OS bind2 folder

Multiple containers could point to the same bind mount on the host OS

Using volumes for storage
- Volumes survive the container lifetime
- Multiple containers can get simultaneous access to the volume
- Data can be stored externally
- Volumes can be used to transition data from one host to another
- Volumes live outside of the container and so don't increase container size
- Volumes use drivers to specify how storage is accessed
    - Enterpirse level drivers are available on the orchestration layer (Kubernetes, Open Shift)

Volume Commands
* `docker volume create myvol` -- use local file systems as storage backend
* `docker volume ls` -- list the volume
* `docker volume inspect myvol` -- shows the properties of the volume
* `docker run -it --name voltest --rm --mount source=myvol,target=/data nginx:latest /bin/sh` --runs the container and attaches to the volume

`sudo ls /var/lib/docker/volumes/myvol/_data`

To simultaneously access files on volumes from multiple containers a special driver is needed.
* Use "readonly" mount option to protect from file locking
* `docker run -it --name voltest --rm --mount source=myvol,target=/data,readonly nginx:latest /bin/sh`
* For non-orchestrated environment can use the local driver NFS type

Managing Storage in SELinux Environment
* The host directory must be writable by the container main process
* If containers are started with a specific UID the number UID can be set.
* `podman inspect image` and look for User to find which user this is.
* Use `sudo chown -R [id]:[id] /hostdir`

Set SELinux Container Context to Host Directory:
* `sudo semanage fcontext -a -t container_file_t "/hostdir(/.*)?" `
    * *container_file_t* - generic way to refer to container context you want to apply
    * writes the *container_file_t* context to the policy but doesn't apply to the filesystem
* write desired context to SELinux policy
    * `sudo restorecon -Rv /hostdir` --applies to filesystem the context

Show the SELinux container context
* `ls -lZd` --Will show *container_file_t* on directory after applying

If user that runs the container is owner of the directory that is going to be bind-mounted, the `:Z` option can be used to automatically set the appropriate SELinux context.
* Recommended approach while using rootless containers
* `podman run -d -v /srv/dbfiles:/var/lib/mysql:Z -e MYSQL_ROOT_PASSWORD=password registry.access.redhat.com/rhscl/mysql-57-rhel7`

A rootless user doesn't have access to the images stored by the root user. This is a different set of images.

# Container Networking

Definitions
* **bridge**: The default networking, allows applications in standalone containers to communicate
* **host**: Removes network isolation between host and containers and allows containers to use the host directly. Available in Swarm with Docker.
* **overlay**: Needing orchestration layer. Allows different Docker daemons to be connected using a software defined network. Allows standalone containers on different Docker hosts to communicate.
* **macvlan**: Assigns a MAC address to a container, making it appear as a physical device on the network. Good for legacy applications.
* **none**: Completely disable networking.
* **plugins**: Third-Party plugins usually seen in orchestration layer.

`ip a | less`
* Docker Bridge listed as docker0 and IP address: 172.17.0.1

Bridge connects to main network (Example: *ens33*) which connects to the outside
* ens33 is a physical network card on the host
* bridge is a logical device

Connected to the bridge are veth devices which are created by the container
Using NAT -Network Address Translation 

Containers can communicate to each other and to the outside but incoming traffic will stop.

Port-forwarding is used to expose containers in a docker environment.

Bridge networking is the default. A container network is created on internal IP address 172.17.0.0/16

Containers will get an IP address in that range when started.

Additional bridge networks can be created

When creating additonal bridge networks, automatic sevice discovery is added so that new containers can be reached by name.

No traffic is allowed between different bridge networks because of namespaces that provide strict isolation.

Can not create routes between different bridge networks.

Bridge is default in Docker and in rootful containers with Podman.

* `docker network ls` -- see default networking
* `docker network inspect bridge` -- Details of network

Start containers on default network
* `docker run -dit --name alpine1 alpine ash`
Verify the containers are started
* `docker container ls`
Check what is happening on the Network Bridge and notice the ip addresses of the containers
* `docker network inspect bridge`
View the container perspective on current network
* `docker attach alpine1; ip addr show`
From here able to ping the bridge IP in docker 172.17.0.1

Create a custom bridge
* `docker network create --driver bridge alpine-net`
* `docker network ls`
* `docker network inspect alpine-net`

Start containers on a specific network. If should be on two, then 2nd network needs to be setup later.
 
 Alpine1 and Alpine setup on custom network alpine-net
* `docker run -dit --name alpine1 alpine --network alpine-net ash`

Have alpine4 connect additionally to the default bridge
* `docker network connect bridge alpine4`

You can ping containers on the same bridge using the container name
* `ping alpine4`

Show available routes
* `ip route show`

## Podman rootless containers

Rootless container cannot get an IP address
* root priviledges required to get IP address

Rootless container also cannot bind to a privileged port

The only way to provide access to rootless container is by configuring port forwarding

In doing inspect of rootless container will not see Network Settings

An inspect of a rootful container will show the Network Settings

`ip a`
* cni-podman -- This is the bridge
* veth -- Container that interfaces with the bridge

Port Forwarding -- Container on 8080 forwarded to 8000 on the host
* `podman run -d -p 8088:8080 nginx`

Allows access only if traffic coming from a specific IP address
* `podman run -d -p 127.0.0.1:8088:8080 nginx`

Find which port mapping applies to a specific container
* `podman ps` also shows this
* `sudo podman port`

In microservices, connected containers may be running on different hosts

In order to directly connect overlay networking is required

Overlay networding is available in orchestration layers like Swarm and Kubernetes

Hosts on a bridge with each hosts having their own containers can communicate using an overly network using SDN
* SDN - software defined networking is implemented with an agent on the different hosts. The containers can then communicate on this network using this orchestration layer.


# Container Orchestration

Kubernetes concerns running containers on the cloud

API Resources - define properties that allow you to store your container in the cloud and to store your configuration in a cloud in such a way that it is decoupled.

You don't have a relation to a specific node in your cloud environment.

These properties that are needed to work with applications in a cloud environment have their result stored in the Kubernetes database -> etcd

Kubernetes manages pods that manages your container and cloud properties

To manage scalability you use the deployment which determines how you run your pods. This also manages availability.

The configmap is a configuration file stored in the etcd

Not dealing with files stored on local servers

Kubernetes can be used as a managed service in public and private cloud
- Amazon EKS
- Google Cloud GKE
- Azure AKS
- OpenStack Magnum

Can be used as an on-premise installation and test-drive platform

Common Distributions:
- Canonical Kubernetes - based on opensource runs on-premise and in cloud
- Rancher -multi-cluster Kubernetes
- Google Anthos
- OpenShift - Developed by RedHat on-premise and cloud. Adds features for developer workflow.

Common Learning Environments
- Minikube - Ubuntu
- Docker Desktop - Window, MacOs
- Microk8s
- K3s
- Kind

Minikube links
* https://minikube.sigs.k8s.io/docs/start/
* https://www.linuxtechi.com/how-to-install-minikube-on-rhel/
* https://minikube.sigs.k8s.io/docs/drivers/podman/

How to deploy
* `kubectl create deploy -h | less`

Create deployment of 3 containers using nginx image
* `kubectl create deployment my-dep --image=nginx --replicas=3`

Show current status of container deployment
* `kubectl get all`

Other Courses
* EX180 specialist in Containers and OpenShift course
* CKAD, CKA certified Kubernetes Developer
* Getting Started with Kubernetes